# Istari AIAA Hands on Keyboard Event

## About this doc

This notebook demonstrates the Istari platform workflow for a Group 3 expendable UAV (tailless flying wing).
You will modify wing design parameters, run an nTop structural analysis, and verify compliance with system requirements.

**Goal**: Achieve the lowest Structure Weight while meeting all requirements.

In [5]:
# Install dependencies
!pip install istari-digital-client -q

## User Data

Enter your credentials and model IDs below.

In [ ]:
#@title User Configuration { display-mode: "form" }
#@markdown **Istari Platform Settings**

ISTARI_ENVIRONMENT_URL = "https://fileservice-v2.stage.istari.app" #@param {type:"string"}
ISTARI_PAT = "your-token-here" #@param {type:"string"}

#@markdown ---
#@markdown **Model IDs**

NTOP_MODEL_ID = "07e0de43-b955-427f-8315-3c11615e84f6" #@param {type:"string"}
INPUT_JSON_MODEL_ID = "3ed80ec5-0072-4f6c-9820-230785eb4055" #@param {type:"string"}

## Parameters

Enter the wing design parameters for your drone here. These values will be sent to nTop for structural analysis.

In [ ]:
# Drone Wing Design Parameters
# Edit these values to change the nTop model inputs

span = 3657.6              # Wingspan (mm) - default ~12ft
le_sweep_p1 = 46.5         # Leading edge sweep - inboard (deg)
le_sweep_p2 = 127.5         # Leading edge sweep - outboard (deg)
te_sweep_p1 = -46.5        # Trailing edge sweep - inboard (deg)
te_sweep_p2 = 15.0         # Trailing edge sweep - outboard (deg)
panel_break_span_pct = 0.31  # Panel break location (fraction of span)

In [8]:
from istari_digital_client.client import Client
from istari_digital_client.configuration import Configuration

# Connect to Istari
client = Client(
    config=Configuration(
        registry_url=ISTARI_ENVIRONMENT_URL,
        registry_auth_token=ISTARI_PAT,
    )
)

# Get the nTop model by ID
ntop_model = client.get_model(NTOP_MODEL_ID)
print(f"Found nTop Model: {ntop_model.display_name or ntop_model.name}")
print(f"  ID: {ntop_model.id}")

# Get the input.json model by ID
input_json_model = client.get_model(INPUT_JSON_MODEL_ID)
print(f"\nFound Input JSON Model: {input_json_model.display_name or input_json_model.name}")
print(f"  ID: {input_json_model.id}")

Found nTop Model: Istari_nTop_Drone_1.0
  ID: 07e0de43-b955-427f-8315-3c11615e84f6


2025-12-23 19:13:17 -  istari_digital_client.v2.api.v2_api:__call__:136 - ERROR - Exception ForbiddenException in method V2Api.get_model : (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'date': 'Tue, 23 Dec 2025 19:13:16 GMT', 'content-length': '156', 'content-type': 'application/json', 'vary': 'Accept-Encoding', 'x-envoy-upstream-service-time': '307', 'server': 'istio-envoy'})
HTTP response body: {"detail":"Permission Denied: \"user:b01bd51f-c992-4e34-9327-386a8b80c2a4 DOES NOT have 'view' permissions to model:db692662-c304-496b-8a90-82c7b63a10da\""}
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/istari_digital_client/log_utils.py", line 30, in wrapper
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/istari_digital_client/v2/api/v2_api.py", line 12368, in get_model
    resp_obj: Model = self.api_client.response_deserialize(
                      ^^^^^^^^^^^^^^^^^^^^

ForbiddenException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'date': 'Tue, 23 Dec 2025 19:13:16 GMT', 'content-length': '156', 'content-type': 'application/json', 'vary': 'Accept-Encoding', 'x-envoy-upstream-service-time': '307', 'server': 'istio-envoy'})
HTTP response body: {"detail":"Permission Denied: \"user:b01bd51f-c992-4e34-9327-386a8b80c2a4 DOES NOT have 'view' permissions to model:db692662-c304-496b-8a90-82c7b63a10da\""}


# Analyze

## Step 1: Build Input

In [ ]:
import json
import tempfile
import os
import time
from istari_digital_client import JobStatusName, NewSource

# Build input.json for drone model with user parameters
input_json = {
    "description": "",
    "inputs": [
        {"description": "", "name": "Span", "type": "real", "units": "mm", "value": span},
        {"description": "", "name": "LE Sweep P1", "type": "real", "units": "deg", "value": le_sweep_p1},
        {"description": "", "name": "LE Sweep P2", "type": "real", "units": "deg", "value": le_sweep_p2},
        {"description": "", "name": "TE Sweep P1", "type": "real", "units": "deg", "value": te_sweep_p1},
        {"description": "", "name": "TE Sweep P2", "type": "real", "units": "deg", "value": te_sweep_p2},
        {"description": "", "name": "Panel Break Span %", "type": "real", "value": panel_break_span_pct},
        {"description": "", "name": "Mesh Export Path", "type": "file_path", "value": "./Outputs/"}
    ],
    "title": "Drone Wing Analysis"
}

print("Drone Parameters:")
print("=" * 40)
for param in input_json["inputs"]:
    if param["type"] != "file_path":
        units = param.get('units', '')
        print(f"  {param['name']}: {param['value']} {units}")

## Step 2: Run Model

In [ ]:
# Update the existing input.json model with new parameters
import logging
from istari_digital_client.exceptions import ConflictException

input_json_content = json.dumps(input_json, indent=2)

with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    f.write(input_json_content)
    temp_input_path = f.name

# Suppress noisy library logging for expected conflicts
logging.getLogger('istari_digital_client').setLevel(logging.CRITICAL)

# Update the existing model instead of creating a new one
try:
    input_json_model = client.update_model(
        model_id=INPUT_JSON_MODEL_ID,
        path=temp_input_path,
    )
    print(f"Updated input.json model: {input_json_model.id}")
    print(f"  Revision: {input_json_model.revision.id}")
except ConflictException:
    print("No parameter changes detected - using existing model revision.")
    input_json_model = client.get_model(INPUT_JSON_MODEL_ID)
finally:
    # Restore logging for other operations
    logging.getLogger('istari_digital_client').setLevel(logging.WARNING)

os.unlink(temp_input_path)

In [ ]:
# Create source reference for the nTop model
source_ntop = NewSource(
    revision_id=ntop_model.revision.id,
    relationship_identifier="ntop_model",
)

# Run the nTop model with our configured inputs
run_job = client.add_job(
    model_id=input_json_model.id,
    function="@ntop:run_model",
    tool_name="ntopcl",
    tool_version="5.30",
    operating_system="RHEL 9",
    sources=[source_ntop],
)

print(f"Run job started: {run_job.id}")
print("Waiting for nTop analysis to complete...")

# Poll until complete
while run_job.status.name not in [JobStatusName.COMPLETED, JobStatusName.FAILED]:
    time.sleep(5)
    run_job = client.get_job(run_job.id)
    print(f"  Status: {run_job.status.name}")

if run_job.status.name == JobStatusName.COMPLETED:
    print("Drone analysis completed successfully!")
else:
    print(f"Drone analysis failed: {run_job.status.name}")

## Step 3: View Results

In [ ]:
# Get the output from the run
input_json_model = client.get_model(input_json_model.id)

print("=" * 60)
print("                  Drone Analysis Results")
print("=" * 60)

# Find and display the output.json
for artifact in input_json_model.artifacts:
    if artifact.name == "output.json":
        content = artifact.read_text()
        output_data = json.loads(content)

        print("\n  OUTPUTS:")
        print("-" * 40)

        # Handle the nTop output format (JSON object inside value.jsonObject)
        if isinstance(output_data, list):
            for item in output_data:
                if isinstance(item, dict) and item.get("type") == "json":
                    json_obj = item.get("value", {}).get("jsonObject", {})
                    for name, value in json_obj.items():
                        print(f"    {name}: {value:.4f}")

print("\n" + "=" * 60)

# Verify

In [ ]:
# =============================================================================
# VERIFICATION AGAINST SYSTEM REQUIREMENTS
# =============================================================================
# Based on Group 3 UAV SysML requirements:
# - Max Takeoff Weight: 500 lb (structure must leave room for fuel, payload, etc.)
# - Structure Weight budget: ~300 lb max (leaving margin for other systems)
# - Fuel Volume: Must be sufficient for 3000+ nm range
# =============================================================================

requirements = {
    "Structure_Weight": {
        "max": 300.0,
        "units": "lb"
    },
    "Fuel_Volume": {
        "min": 2500.0,  # placeholder - adjust based on actual outputs
        "units": "mm³"
    }
}

# Extract the output values from the last run
output_values = {}
for artifact in input_json_model.artifacts:
    if artifact.name == "output.json":
        content = artifact.read_text()
        output_data = json.loads(content)

        # Handle nTop output format (JSON object inside value.jsonObject)
        if isinstance(output_data, list):
            for item in output_data:
                if isinstance(item, dict) and item.get("type") == "json":
                    json_obj = item.get("value", {}).get("jsonObject", {})
                    output_values = json_obj

# Map outputs
fuel_volume = output_values.get("Fuel_Volume")
structure_weight = output_values.get("Structure_Weight")
surface_area = output_values.get("Surface_Area")

# Run verification
print("=" * 60)
print("              DRONE VERIFICATION RESULTS")
print("=" * 60)

# Check structure weight constraint
weight_ok = structure_weight is not None and structure_weight <= requirements["Structure_Weight"]["max"]

print(f"\n  Structure Weight:")
print(f"    Value:    {structure_weight:.2f} lb" if structure_weight else "    Value:    N/A")
print(f"    Max:      {requirements['Structure_Weight']['max']} lb")
print(f"    Status:   {'✅ PASS' if weight_ok else '❌ FAIL (exceeds max)'}")

# Check fuel volume constraint
fuel_ok = fuel_volume is not None and fuel_volume >= requirements["Fuel_Volume"]["min"]

print(f"\n  Fuel Volume:")
print(f"    Value:    {fuel_volume:.2f} mm³" if fuel_volume else "    Value:    N/A")
print(f"    Min:      {requirements['Fuel_Volume']['min']} mm³")
print(f"    Status:   {'✅ PASS' if fuel_ok else '❌ FAIL (below min)'}")

print(f"\n  Surface Area:")
print(f"    Value:    {surface_area:.2f} mm²" if surface_area else "    Value:    N/A")
print(f"    (Info only - affects aerodynamic performance)")

# Overall result
all_pass = weight_ok and fuel_ok

print("\n" + "=" * 60)
if all_pass:
    print(f"  ✅ ALL REQUIREMENTS PASSED!")
    print(f"  🎯 Your Structure Weight: {structure_weight:.2f} lb")
    print(f"     Can you get it lower while still passing?")
else:
    print("  ❌ REQUIREMENTS NOT MET")
    if not weight_ok:
        print("     - Structure Weight exceeds maximum!")
    if not fuel_ok:
        print("     - Fuel Volume below minimum!")
print("=" * 60)